In [1]:
import importlib

import torch
import torch.nn as nn
import torch.optim as optim
 
from constants import *
import utils
importlib.reload(utils)

CLASSIFIED_STATES = ['boredom', 'flow', 'frustration', 'neutral']

HIDDEN_SIZE = len(DATA_COLUMNS)
OUTPUT_SIZE = len(CLASSIFIED_STATES)

In [42]:
class MentalStateClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.input = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)    
    self.output = nn.Linear(HIDDEN_SIZE, OUTPUT_SIZE)
    
    self.softmax = nn.Softmax(dim=1)
    self.dropout = nn.Dropout(p=0.2)
    self.relu = nn.ReLU()

  def forward(self, x): # (1, 25, 40) -> (1, 4)
    x = x.mean(dim=1) # Average 1 second of data 
    
    x = self.dropout(x)
    x = self.input(x)
    x = self.relu(x) 
    x = self.output(x)
    x = self.softmax(x)

    if not self.training:
      x = torch.round(x, decimals=3)
    return x
    
model = MentalStateClassifier()
 
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
n_epochs = 2
batch_size = 8
n_batches = utils.getTrainingRowNum(CLASSIFIED_STATES) // SAMPLING_RATE // batch_size - 20

In [44]:
import copy
import tqdm
import numpy as np
importlib.reload(utils)

best_acc = - np.inf
best_weights = None
train_loss_hist = []
train_acc_hist = []
test_loss_hist = []
test_acc_hist = []
X_batch = None # for exporting to ONNX

for epoch in range(n_epochs):
  epoch_loss = []
  epoch_acc = []
  model.train() # set model to training mode
  data_loader = utils.lazy_load_training_data(states=CLASSIFIED_STATES)

  X_train, y_train = next(data_loader)

  with tqdm.trange(n_batches, unit="batch", mininterval=0) as bar:
    bar.set_description(f"Epoch {epoch}")
    j = 0
    for i in bar:
      start = j * batch_size
      if start + batch_size >= len(X_train):
        j = 0
        start = 0
        X_train, y_train = next(data_loader)
      
      j += 1
      X_batch = X_train[start:start+batch_size]
      y_batch = y_train[start:start+batch_size] 

      # forward pass
      y_pred = model(X_batch)
      loss = loss_fn(y_pred, y_batch)
      
      # backward pass
      optimizer.zero_grad()
      loss.backward()
      # update weights
      optimizer.step()
      # compute and store metrics
      acc = (torch.argmax(y_pred) == torch.argmax(y_batch)).float().mean()
      epoch_loss.append(float(loss))
      epoch_acc.append(float(acc))
      bar.set_postfix(
          loss=float(loss),
          acc=float(acc)
      )
  model.eval()
  X_test, y_test = utils.load_testing_data(states=CLASSIFIED_STATES)
  y_pred = model(X_test)
  ce = loss_fn(y_pred, y_test)
  acc = (torch.argmax(y_pred, 1) == torch.argmax(y_test, 1)).float().mean()
  ce = float(ce)
  acc = float(acc)
  train_loss_hist.append(np.mean(epoch_loss))
  train_acc_hist.append(np.mean(epoch_acc))
  test_loss_hist.append(ce)
  test_acc_hist.append(acc)
  if acc > best_acc:
      best_acc = acc
      best_weights = copy.deepcopy(model.state_dict())
  print(f"Epoch {epoch} validation: Cross-entropy={round(ce, 4)}, Accuracy={round(acc, 4)}")

model.load_state_dict(best_weights)

Epoch 0: 100%|██████████| 2896/2896 [00:20<00:00, 141.63batch/s, acc=1, loss=1.49] 


Epoch 0 validation: Cross-entropy=1.4833, Accuracy=0.2605


Epoch 1: 100%|██████████| 2896/2896 [00:20<00:00, 140.08batch/s, acc=1, loss=1.37] 


Epoch 1 validation: Cross-entropy=1.4835, Accuracy=0.2603


<All keys matched successfully>

In [45]:
torch.round(model(X_batch), decimals=3), X_batch.shape

(tensor([[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]], grad_fn=<RoundBackward1>),
 torch.Size([8, 25, 40]))

In [47]:
# %pip install onnx onnxscript
# %pip install onnxruntime

torch.onnx.export(model,                       # model being run
                  X_batch,                         # model input (or a tuple for multiple inputs)
                  './exported_models/bnnm-NN-001.onnx',            # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['X'],       # the model's input names
                  output_names = ['Y']       # the model's output names
                  )